In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Day 10 - MATH/PHYS 241

There is a very large class of problems in the physical sciences in which a conservative property moves through space at a rate proportional to some gradient (i.e., it follows a first-order rate law). Here, “first order” refers to an equation that contains the first derivative but no higher derivatives.

This is different from the use in chemistry where it denotes a reaction rate proportional to the first power of a concentration (or on Day 09 during our discussion of first order loss processes with box modeling). The conservative property that flows can be the moles of ions in a solution, or the thermal energy of atoms in a material, or the mass of [regolith](https://en.wikipedia.org/wiki/Regolith) on a hillside.

In the study of transport phenomena, the amount of the property in question that flows per unit area per unit time is called a flux ($q$). In the first case (moles of ions in a solution), the quantity of ions would move in proportion to the concentration gradient of those ions; in the second case (thermal energy), the gradient is in temperature; and in the third case, the gradient is the slope of the hill.

Regardless of the property in question or the gradient it flows down, all problems of this class acquire the same mathematical form and can be solved in a similar manner.

# 1-D Diffusion

The [diffusion equation](https://en.wikipedia.org/wiki/Diffusion_equation) will look a little different depending upon exactly what is being transported and the number of dimensions the transport takes place over. However, for 1-D diffusion, it will resemble [Fick's 2nd Law](https://en.wikipedia.org/wiki/Fick%27s_laws_of_diffusion#Fick's_second_law):

$$
\begin{align}
\frac{\partial \varphi}{\partial t} &= D \frac{\partial ^2 \varphi}{\partial x^2} \tag{1}
\end{align}
$$

Where:
* $\varphi$ is a function, $\varphi (x,t)$, that describes the concentration ($\frac{\text{mol}}{\text{m}^3}$) everywhere at time $t$
* $t$ is time
* $D$ is the diffusion constant ($\frac{\text{m}^2}{\text{s}}$)
* $x$ is the position.

The equation describes how the concentration will change over time and space.

If we have temperature diffusion instead of chemical, we just need to swap $\varphi$ for $T$ and note that $D$ now represents the *thermal diffusivity constant*, $\alpha$ for the substance the heat is diffusing through.

$$
\begin{align}
\alpha = \frac{k}{c\rho} \tag{2}
\end{align}
$$

Where $k$ is the [thermal conductivity](https://en.wikipedia.org/wiki/Thermal_conductivity_and_resistivity) of the medium, $c$ is the specific heat capacity of the material, and $\rho$ is the density.

# Example: Temperature Diffusion in a Rod

Let's say we have an thin iron rod with a length of $1 \ \text{m}$. For simplicity, let's assume the sides of the rod are fully insulated so no heat may escape through the sides. Let's also make our rod a 'donut' shape, so the ends are touching. This seems kind of silly, but it actually makes our problem easier to deal with (see boundary conditions below).

Let the entire length of the rod be at a uniform $10 ^\circ \text{C}$. However, let's bring a small section in the very middle of the rod of the rod magically up to $100 ^\circ \text{C}$.

**How is the temperature of the rod going to evolve over time?**

You can already guess the middle "hot" section gets cooler and the rest of the rod will get warmer, but what does this process look like and how long does it take to reach its equilibrium temperature?

These are difficult questions to answer, but we can model the heat flow using python.

## Descretization

The first step to solving our diffusion problem is to *descretize* the system. Consider a domain in $x, t$ space as the region in which we seek a solution to our 1-D diffusion equation. In reality the solution is a continuous surface, but in numerical solutions the space-time plane is discretized into a set of points.

The points do not have to be at equal intervals, but for simplicity we will take the space step as a constant $\Delta x$ and the time step as a constant $\Delta t$.

Thus, points in $x$ and $t$ lie at $x = j\Delta x$ and $n\Delta t$ where $j = 1, 2, 3, ..., J$ (with $J$ the maximum value of $j$) and $n = 1, 2, 3, ... , N$ (with $N$ the maximum value of $n$).

So, if we have a total length $L$, by choosing a spacial step size of $\Delta x$, we see $J = \frac{L}{\Delta x}$. Similar for time, if the total time is $t_{end}$, by choosing a temporal step size of $\Delta t$, we see $N = \frac{t_{end}}{\Delta t}$.

You can think of chunking the bar into small cells of width $\Delta x$. What we want to know is how each individual cell changes in temperature as we add small time steps, $\Delta t$, until we reach $t_{end}$.

## Difference Opperators

We covered numerical derivatives in a previous notebook. To solve the heat equation numerically, we use something called forward-time-centered-space scheme ([FTCS](https://en.wikipedia.org/wiki/FTCS_scheme#:~:text=In%20numerical%20analysis%2C%20the%20FTCS,applied%20to%20the%20heat%20equation.)). The scheme uses the forward Euler Method in time and the Central Difference method in space to solve the partial differential equation. Note that the Central Difference method is applied over a 2nd derivative.

This is the preferred scheme for the 2nd order partial derivative associated with the diffusion equation due to its stability. Applying the forward differencing method unfortunately results in a non-physical result due to the way it propogates errors.

If we apply the methods to their respective sides in equation 1, we see:

$$
\begin{align}
\frac{T_j^{n+1} - T_j^n}{\Delta t} &= \alpha \frac{T_{j+1}^n - 2T_j^n + 2T_{j-1}^n}{\Delta x^2} \tag{3}
\end{align}
$$

We just need to do a little algebra to solve the equation for $T_j^{n+1}$ :

$$
\begin{align}
{T_j^{n+1}} &= T_j^n + \alpha \frac{\Delta t}{\Delta x^2} \left( {T_{j+1}^n - 2T_j^n + 2T_{j-1}^n} \right) \tag{4}
\end{align}
$$

Equation 4 gives the temperature of cell $j$ at the next time interval $n+1$.

## Stability

A choice of finite difference schemes begs the question of which scheme is better. Better can be defined in a number of ways, but we define it as a scheme that is accurate, efficient, and easy.

A scheme is accurate (also called convergent) if its solution approaches the analytic solution as the discretization steps are reduced in size.

A scheme is efficient if it minimizes computation time.

Easy refers to our ability to comprehend and code the scheme.

Judging a scheme's efficiency and ease of use is straightforward, but how do we guarantee its accuracy? The answer is that we must guarantee its consistency and stability.

A scheme's system of algebraic equations is consistent if, as $\Delta x$ and $\Delta t$ approach $0$, the system becomes equivalent to the differential equations at each grid point.

A scheme is stable if spontaneous perturbations (such as round-off error) in the solution of the algebraic equations decay as the computations proceed.

Finally, a solution of the algebraic equations approximating a differential equation is convergent if the approximate solution approaches the exact solution as grid size tends to zero.

The FTCS scheme is stable under certain conditions. Our explicit FTCS scheme (equation 4), is stable if:

$$
\begin{align}
\Delta t \leq \frac{\Delta x^2}{2 \alpha} \tag{5}
\end{align}
$$

Even with the above statement true, your result can still have numerical errors. Error analysis becomes a major component of validating any numerical model.

## Boundary Conditions

The last thing we need to discuss before we attempt to implement the above methods is our treatment of boundary conditions.

For example, let's say we had left our bar as a straight piece of metal. Assuming the middle is still fully insulated, heat could transfer in or out of the metal through the ends.

This presents a potential problem when we solve our equation as we need to know how to treat heat diffusing into/out of the ends.

For instance, we could "touch" the front end to an infinite heat reservoir that holds a constant temperature. Assuming the bar is initially cooler than the heat reservoir, the end cell will rise to the temperature of the heat reservoir, then stay at this elevated temperature. After each successive pass of our FTCS scheme, we would need to update the temperature of this end cell so it *stays* at the boundary condition temperature, otherwise it could lose heat, which doesn't make sense.

Boundary conditions have to be treated on a case-by-case basis, so be aware!

# Class Problem 1

Going back to our initial problem with a thin iron rod of length $1 \ \text{m}$ with the ends connected. Let the entire length of the rod be at a uniform $10 ^\circ \text{C}$.

1. Descretize the rod using a length step $\Delta x = 1 \ \text{mm}$
2. Set the middle $1 \ \text{cm}$ of the rod to $100 ^\circ \text{C}$.
3. Create a plot of the temperature vs location of the rod.

To assist with implementing the FTCS scheme for our problem, I have included a function that will correctly apply the boundary conditions required.

In [ ]:
def diffusion(T, del_t, del_x, a):

    for i in range(len(T)):
        if i >= 1 and i < len(T)-1:
            T_new[i] = T[i] + del_t/((del_x)**2) * a * (T[i+1] - 2*T[i] + T[i-1])

        # periodic boundary:
        T_new[0] = T[0] + del_t/((del_x)**2) * a * (T[1] - 2*T[0] + T[-1])
        T_new[-1] = T[-1] + del_t/((del_x)**2) * a * (T[0] - 2*T[-1] + T[-2])

    return T_new

Note that the thermal diffusivity constant for iron is $\alpha = 23 \frac{\text{mm}^2}{\text{s}}$.

4. Calculate the maximum time step $\Delta t$ we can use to have a stable result.
5. Apply `diffusion` and calculate the evolution of the temperature profile over 10 seconds. Does the profile match with what you were expecting based on your understanding of heat flow (even if you haven't had physics)?
6. At what temperature is thermal equilibrium (this can be calculated using $Q = mC\Delta T$, or you can ask Terry)? Allow you code to time step until the bar reaches this approximate temperature. How long did it take?

# Class Problem 2

**Salt Around a Dissolving Sphere**

Consider a sphere of [halite](https://en.wikipedia.org/wiki/Halite) of radius $r$, immersed in an infinite still ocean that, at $t_0 = 0$ is freshwater.

Model the concentration of dissolved salt as a function of radial distance $r$, away from the sphere, at time $t$.

This can be done by making multiple plots of the instantaneous salt concentration between $0$ and $r$ ($[C]$ vs $r$) or by plotting how the concentration at particular different distances will change over time ($[C]$ vs $t$).

You will want to utilize `diffusion` and implement new boundary conditions.

Use a diffusion constant $D = 0.1 \ \frac{\text{m}^2}{\text{s}}$.